# 2023 CITS4012 Assignment
*Make sure you change the file name with your student id.*

# Readme
*If there is something to be noted for the marker, please mention here.* 

*If you are planning to implement a program with Object Oriented Programming style, please check the bottom of the this ipynb file*

In [ ]:
import sys
sys.path.append('./src/QA')
from importlib import reload
from data_prep import DataPrep
reload(data_prep)
from data_prep import DataPrep

ModuleNotFoundError: ignored

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

In [1]:
# only need to run these commands if using Colab
# Mount google drive and change directory to the git repo folder
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/CITS4012_2023
import sys
# colab seems to require an explicit path for this to work from mounted drive
sys.path.append('/content/gdrive/MyDrive/CITS4012_2023/src/QA')

Mounted at /content/gdrive
/content/gdrive/MyDrive/CITS4012_2023


In [ ]:
# when not using colab
import sys
sys.path.append('./src/QA')

In [2]:
from data_prep import DataPrep

In [3]:
df = DataPrep.parse_tsv('WikiQA-train.tsv')
question_id_list = df.loc[:, "QuestionID"]
question_id_list = list(set(question_id_list))
question_id_list.sort() # fix sorting or dont sort

question_id = {}
for i in df.index:
    row = df.loc[i]
    id = row["QuestionID"]
    if id not in question_id:
        question_id[id] = {"document": []}
        question_id[id]["question"] = row["Question"]
    
    question_id[id]["document"].append((row["Sentence"], row["Label"]))

In [5]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords as sw
stop_words = sw.words()
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [12]:
def word_match(q_tokens, doc):
    # remove stop words from question so we are only matching important words
    filtered_sentence = [word for word in q_tokens if not word in stop_words]
    print(filtered_sentence)
    # lemmatise question words remaining
    q_lemmas = [lemmatizer.lemmatize(word) for word in filtered_sentence]
    print(q_lemmas)
    # compare word in document to see if they match a word
    word_match_list = []
    for sentence in doc:
        for word in sentence:
            word_lemma = lemmatizer.lemmatize(word)
            if word_lemma in q_lemmas:
                word_match_list.append(1)
            else:
                word_match_list.append(0)
    # output
    return word_match_list


['glacier', 'caves', 'formed', '?']
['glacier', 'cave', 'formed', '?']
glacier
cave
glacier
cave
glacier
cave
cave
formed
glacier
caves
caves
caves


In [4]:
toks = DataPrep.tokenize_question_and_doc(question_id["Q1"])
toks


(['how', 'are', 'glacier', 'caves', 'formed', '?'],
 [['A',
   'partly',
   'submerged',
   'glacier',
   'cave',
   'on',
   'Perito',
   'Moreno',
   'Glacier',
   '.'],
  ['The', 'ice', 'facade', 'is', 'approximately', '60', 'm', 'high'],
  ['Ice', 'formations', 'in', 'the', 'Titlis', 'glacier', 'cave'],
  ['A',
   'glacier',
   'cave',
   'is',
   'a',
   'cave',
   'formed',
   'within',
   'the',
   'ice',
   'of',
   'a',
   'glacier',
   '.'],
  ['Glacier',
   'caves',
   'are',
   'often',
   'called',
   'ice',
   'caves',
   ',',
   'but',
   'this',
   'term',
   'is',
   'properly',
   'used',
   'to',
   'describe',
   'bedrock',
   'caves',
   'that',
   'contain',
   'year',
   '-',
   'round',
   'ice',
   '.']],
 [['OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA'],
  ['OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA'],
  ['OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA'],
  ['BOA',
   'IOA',
   'IOA',
   'IOA',
   'IOA',
   'IOA',
   'IOA',
   

In [ ]:
question_id["Q1"]

{'document': [('A partly submerged glacier cave on Perito Moreno Glacier .',
   0),
  ('The ice facade is approximately 60 m high', 0),
  ('Ice formations in the Titlis glacier cave', 0),
  ('A glacier cave is a cave formed within the ice of a glacier .', 1),
  ('Glacier caves are often called ice caves , but this term is properly used to describe bedrock caves that contain year-round ice.',
   0)],
 'question': 'how are glacier caves formed?'}

questions that NER tagging works in:
- how many series in hockey before the stanley cup
- how much are the harry potter movies worth

questions it doesn't work in:
- how did anne frank die
- what did benedict arnold die
- how old is kurt cobain's daughter


# 2.QA Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

# 3.Model Testing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

###3.1. Input Embedding Ablation Study

(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 



In [ ]:
from word_embed import WordEmbed
from feat_extract import FeatExt
from embed_doc import EmbedAndConcat

[=================================================-] 99.4% 127.4/128.1MB downloaded

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
full_embedded_input = {}
# "question" = list of np.arrays for each word
# "document" = a list of embedded sentences, with each sentence having one array for each word
# "answer" = list of sentences, each sentence a list of answer labels
for q_id in question_id:
  try:
    tokens = DataPrep.tokenize_question_and_doc(question_id[q_id])
    answer = tokens[2]
    q_lemmas = FeatExt.lemma_the_question(tokens[0])
    q_ner_tags = FeatExt.ner_tag(question_id[q_id]["question"])
    q_embedded = EmbedAndConcat.q_concat(tokens[0], with_ner=True, question_ner_tags=q_ner_tags)
    doc_ner_tags = FeatExt.ner_tag_doc(question_id[q_id]["document"])
    doc_embedded = EmbedAndConcat.doc_concat(tokens[1], with_ner=True, doc_ner_tags=doc_ner_tags,
                                             q_lemmas=q_lemmas)
    full_embedded_input[q_id] = {"question": q_embedded, "document": doc_embedded, "answer": answer}
  except:
    print(q_id)
    break

In [ ]:
no_ner_embedded_input = {}
for q_id in question_id:
    tokens = DataPrep.tokenize_question_and_doc(question_id[q_id])
    answer = tokens[2]
    q_lemmas = FeatExt.lemma_the_question(tokens[0])
    q_embedded = EmbedAndConcat.q_concat(tokens[0])
    doc_embedded = EmbedAndConcat.doc_concat(tokens[1], q_lemmas=q_lemmas)
    full_embedded_input[q_id] = {"question": q_embedded, "document": doc_embedded, "answer": answer}

###3.2. Attention Ablation Study
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

###3.3. Hyper Parameter Testing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 